In [3]:
import os
import hashlib
import marshal
from pyeoskit import eosapi, wallet, db
from pyeoskit import config
from pyeoskit._hello import _eosapi

db.reset()
config.main_token = 'UUOS'

if os.path.exists('test.wallet'):
    os.remove('test.wallet')
psw = wallet.create('test')

wallet.import_key('test', '5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc')

def publish_contract(account_name, code, abi):
    m = hashlib.sha256()
    code = compile(code, "contract", 'exec')
    code = marshal.dumps(code)
    m.update(code)
    code_hash = m.hexdigest()
    r = eosapi.get_code(account_name)
    if code_hash != r['code_hash']:
        eosapi.set_contract(account_name, code, abi, 1)
    return True

#eosapi.set_nodes(['https://nodes.uuos.network:8443'])
eosapi.set_nodes(['http://127.0.0.1:8888'])

def run_test_code(code, account_name='helloworld11'):
    publish_contract(account_name, code, abi)
    try:
        r = eosapi.push_action(account_name, 'sayhello', b'hello,world', {account_name:'active'})
        print(r['processed']['action_traces'][0]['console'])
    except Exception as e:
        print(e)


def set_code(account_name, code):
    m = hashlib.sha256()
    code = compile(code, "contract", 'exec')
    code = marshal.dumps(code)
    m.update(code)
    code_hash = m.hexdigest()
    r = eosapi.get_code(account_name)
    if code_hash == r['code_hash']:
        return

    setcode = {"account":account_name,
               "vmtype": 1,
               "vmversion":0,
               "code":code.hex()
               }
    eosapi.push_action('eosio', 'setcode', setcode, {account_name:'active'})
    
    return True

def set_abi(account, abi):
    db.set_abi(account, abi)
    abi = _eosapi.pack_abi(abi)
    setabi ={'account':account, 'abi':abi.hex()}
    eosapi.push_action('eosio', 'setabi', setabi, {account:'active'})

print('done!')

done!


### store_i64

```python
def store_i64(scope: Union[uint64_t, str], table: Union[uint64_t, str], payer: Union[uint64_t, str], id: Union[uint64_t, str],  data: bytes) -> int32_t
```

In [221]:
code = '''
import db
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    id = 'hello'
    itr = db.store_i64(scope, table, payer, id, b'world')
    print(itr)
'''
abi = ''
name = 'helloworld11'
publish_contract(name, code, abi)
try:
    r = eosapi.push_action(name, 'sayhello', b'hello,world', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)

0



### find_i64

```python
def find_i64(code: Union[uint64_t, str], scope: Union[uint64_t, str], table: Union[uint64_t, str], id: Union[uint64_t, str]) -> int32_t
```

In [225]:
code = '''
import db
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    id = 'hello'
    itr = db.find_i64(code, scope, table, id)
    print(itr)
'''
abi = ''
name = 'helloworld11'
publish_contract(name, code, abi)
try:
    r = eosapi.push_action(name, 'sayhello', b'hello,world', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)

0



### get_i64

```python
def get_i64(iterator: int32_t) -> bytes
```

In [227]:
code = '''
import db
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    id = 'hello'

    itr = db.find_i64(code, scope, table, id)
    print(itr)
    data = db.get_i64(itr)
    print(data)
'''
abi = ''
name = 'helloworld11'
publish_contract(name, code, abi)
try:
    r = eosapi.push_action(name, 'sayhello', b'hello,world', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)
    

0
b'world'



### update_i64

```python
def update_i64(iterator: int32_t, payer: Union[uint64_t, str], data: bytes) -> None
```

In [231]:
code = '''
import db
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    id = 'hello'
    data = read_action_data()
    print(data)
    
    itr = db.find_i64(code, scope, table, id)
    print(itr)
    if itr < 0:
        db.store_i64(scope, table, payer, id, b'world')
    else:
        old_data = db.get_i64(itr)
        print(old_data)
        db.update_i64(itr, receiver, data)
'''
abi = ''
name = 'helloworld11'
publish_contract(name, code, abi)
try:
    r = eosapi.push_action(name, 'sayhello', b'hello,worldddd', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)

b'hello,worldddd'
0
b'hello,worldddd'



### lowerbound_i64

```python
def lowerbound_i64(code: Union[uint64_t, str], scope: Union[uint64_t, str], table: Union[uint64_t, str], id: Union[uint64_t, str]) -> int32_t
```

### upperbound_i64

```python
def upperbound_i64(code: Union[uint64_t, str], scope: Union[uint64_t, str], table: Union[uint64_t, str], id: Union[uint64_t, str]) -> int32_t
```

In [237]:
abi = ''
code = '''
import db
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    id = 'hello'
    data = read_action_data()
    if action == N('destroy'):
        while True:
            itr = db.end_i64(code, scope, table)
            if itr == -1:
                break
            itr, primary = db.previous_i64(itr)
            if itr < 0:
                break
            db.remove_i64(itr)
        print('destroy done!')
    elif action == N('init'):
        db.store_i64(scope, table, payer, 1, b'1')
        db.store_i64(scope, table, payer, 3, b'3')
        db.store_i64(scope, table, payer, 5, b'5')
        db.store_i64(scope, table, payer, 7, b'7')
        print('init done!')
    elif action == N('test'):
        itr = db.upperbound_i64(code, scope, table, 3)
        data = db.get_i64(itr)
        print('upperbound of 3:', data)

        itr = db.upperbound_i64(code, scope, table, 2)
        data = db.get_i64(itr)
        print('upperbound of 2:', data)

        itr = db.lowerbound_i64(code, scope, table, 3)
        data = db.get_i64(itr)
        print('lowerbound of 3:', data)

        itr = db.lowerbound_i64(code, scope, table, 2)
        data = db.get_i64(itr)
        print('lowerbound of 2:', data)
'''

name = 'helloworld11'
publish_contract(name, code, abi)
try:
    r = eosapi.push_action(name, 'destroy', b'', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])

    r = eosapi.push_action(name, 'init', b'hello,world', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)


destroy done!

init done!



In [233]:
try:
    r = eosapi.push_action(name, 'test', b'', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)


upperbound of 3: b'5'
upperbound of 2: b'3'
lowerbound of 3: b'3'
lowerbound of 2: b'3'




### next_i64

```python
def next_i64(iterator: int32_t) -> [int32_t, uint64_t]
```

### previous_i64

```python
def previous_i64(iterator: int32_t) -> Union[int32_t, uint64_t]
```


### end_i64

```python
def end_i64(code: Union[uint64_t, str], scope: Union[uint64_t, str], table: Union[uint64_t, str]) -> int32_t
```

### remove_i64

```python
def remove_i64(iterator: int32_t) -> None
```

In [14]:
abi = ''
code = '''
import db
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    id = 'hello'
    data = read_action_data()
    if action == N('destroy'):
        while True:
            itr = db.end_i64(code, scope, table)
            if itr == -1:
                break
            itr, primary = db.previous_i64(itr)
            if itr < 0:
                break
            db.remove_i64(itr)
        print('destroy done!')
    elif action == N('init'):
        db.store_i64(scope, table, payer, 1, b'1')
        db.store_i64(scope, table, payer, 3, b'3')
        db.store_i64(scope, table, payer, 5, b'5')
        db.store_i64(scope, table, payer, 7, b'7')
        print('init done!')
    elif action == N('test'):
        itr = db.find_i64(code, scope, table, 3)
        data = db.get_i64(itr)
        print('++++get_i64:', data)
        print(itr)
        itr_next, primary = db.next_i64(itr)
        print(itr_next)

        data = db.get_i64(itr_next)

        print('db.next_i64: ', data)
        
        itr_previous, primary = db.previous_i64(itr)
        data = db.get_i64(itr_previous)
        print('db.previous_i64: ', data)
'''

name = 'helloworld11'
publish_contract(name, code, abi)


True

In [15]:
try:
    r = eosapi.push_action(name, 'destroy', b'', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])

    r = eosapi.push_action(name, 'init', b'hello,world', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])
except Exception as e:
    print(e)

destroy done!

343
init done!

242


In [16]:
try:
    r = eosapi.push_action(name, 'test', b'', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])
except Exception as e:
    print(e)

++++get_i64: b'3'
0
1
db.next_i64:  b'5'
db.previous_i64:  b'1'

352
